In [25]:
import numpy as np
from numpy import pi
import src.disclination as disc
import matplotlib.pyplot as plt
from importlib import reload
reload(disc)

<module 'src.disclination' from '/home/mark/Dropbox/VS_Code_Projects/tci_disclinations/src/disclination.py'>

In [26]:
# Parameters
norb = 8

nx = 10
nz = 14

n_tot = norb * nx * nx * nz

mass = 2
phs_mass = 2
hoti_mass = 0.0

flux = 2 * pi

test_unitarity = False
plot_am_evals = False
calculate_commutator = False


In [27]:
from time import time

# Generate C4 symmetric basis
rot_op = disc.rotation_matrix(nx, nz)

quadrant_states_basis = np.zeros((n_tot, n_tot // 4), dtype=complex)
c4_basis = np.zeros((n_tot, n_tot), dtype=complex)

for xx in range( nx // 2):
    for yy in range( nx // 2):
        for zz in range(nz):
            for ii in range(norb):
                site_ind = ii + xx * norb + yy * nx * norb + zz * nx * nx * norb
                basis_ind = ii + xx * norb + yy * nx // 2 * norb + zz * nx * nx // 4 * norb
                quadrant_states_basis[site_ind, basis_ind] = 1

for rr in range(4):
    temp = quadrant_states_basis
    for mm in range(4):
        if mm != 0:
            temp = np.exp(1j * rr * pi / 2) * rot_op @ temp
        c4_basis[:, rr * n_tot // 4:(rr + 1) * n_tot // 4] += temp / 2

In [28]:
# Build Hamiltonian, add flux in a C4 symmetric manner
h = disc.defect_free_hamiltonian(nx, nz, mass, phs_mass, hoti_mass)
h = np.reshape(h, (nz, nx, nx, norb, nz, nx, nx, norb))

phi = 1j * flux

for zz in range(nz):
    for ii in range(nx // 2):
        h[zz, ii, nx // 2,     :, zz, ii, nx // 2 - 1, :] *= np.exp(phi / 4)
        h[zz, ii, nx // 2 - 1, :, zz, ii, nx // 2,     :] *= np.exp(-phi / 4)

        h[zz, ii + nx // 2, nx // 2,     :, zz, ii + nx // 2, nx // 2 - 1, :] *= np.exp(-phi / 4)
        h[zz, ii + nx // 2, nx // 2 - 1, :, zz, ii + nx // 2, nx // 2,     :] *= np.exp(phi / 4)
        
        h[zz, nx // 2,     ii, :, zz, nx // 2 - 1, ii, :] *= np.exp(-phi / 4)
        h[zz, nx // 2 - 1, ii, :, zz, nx // 2,     ii, :] *= np.exp(phi / 4)

        h[zz, nx // 2,     ii + nx // 2, :, zz, nx // 2 - 1, ii + nx // 2, :] *= np.exp(phi / 4)
        h[zz, nx // 2 - 1, ii + nx // 2, :, zz, nx // 2,     ii + nx // 2, :] *= np.exp(-phi / 4)

h = np.reshape(h, (norb * nz * nx * nx, norb * nz * nx * nx))

In [29]:
# Check for symmetry
if calculate_commutator:
    commutator = rot_op @ h - h @ rot_op
    print(f'Commutator maximum value: {np.max(np.abs(commutator))}')

    if not np.isclose(0, np.max(np.abs(commutator))):
        plt.imshow(np.abs(commutator))

In [30]:
if test_unitarity:
    print(f'Unitarity check (small is good): {np.max(np.abs(np.identity(n_tot) - c4_basis.conj().T @ c4_basis))}')

if plot_am_evals:
    print('Plotting C4 eigenvalues')
    plt.plot(np.angle(np.diag(c4_basis.conj().T @ rot_op @ c4_basis) * np.exp(1j * 1e-14))/ (pi), 'b.')
    plt.show()

In [31]:
# Rotate Hamiltonian into a C4 eigenvalue basis, separate out blocks
h_rot = c4_basis.conj().T @ h @ c4_basis
h_c4_sectors = [h_rot[ii * n_tot // 4: (ii + 1) * n_tot // 4, ii * n_tot // 4: (ii + 1) * n_tot // 4] for ii in range(4)]

In [32]:
# Calculate density in each sector
u_c4_sectors = []
v_c4_sectors = []

for mat in h_c4_sectors:
    temp_u, temp_v = np.linalg.eigh(mat)
    u_c4_sectors.append(temp_u)
    v_c4_sectors.append(temp_v)

def calculate_rho(evals, evecs):
    rho = np.zeros((nz, nx * nx // 4))

    for ii, energy in enumerate(evals):
        if energy <= 0:
            wf = evecs[:, ii]
            prob_density = np.multiply(np.conj(wf), wf)
            temp_rho = np.reshape(prob_density, (nz, nx * nx // 4, norb))
            rho += np.sum(temp_rho, axis=-1).real

    return rho - norb // 2
    # return rho

rho_c4_sectors = [calculate_rho(u_c4_sectors[ii], v_c4_sectors[ii]) for ii in range(4)]

In [33]:
# Sum charge density over top half of the crystal, calculate bound angular momentum
# Goal is to have summed_angular_momentum = 0.5 for a 2pi flux
c4_evals = [0, -1, -2, -3]
rho_summed_c4_sectors = [np.sum(rho_c4_sectors[ii][:nz//2]) for ii in range(4)]
surf_angular_momentum = np.sum([c4_evals[rr] * rho_summed_c4_sectors[rr] for rr in range(4)])
print(f'Surface AM: {surf_angular_momentum}')

Surface AM: -0.23834198252261185
